Similarities_btw_emails.ipynb file to find the similarity between documents

In [69]:
import os
from collections import Counter
from email.parser import Parser
import math

In [70]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import gensim
from gensim.models.doc2vec import TaggedDocument

In [71]:
rootdir1 = r"C:\Users\rohit\Downloads\enron_mail_20150507.tar\enron_mail_20150507\maildir\lay-k\all_documents\33_"

In [72]:
rootdir2 = r"C:\Users\rohit\Downloads\enron_mail_20150507.tar\enron_mail_20150507\maildir\lay-k\all_documents\34_"

In [73]:
with open(rootdir1,"r") as f:
    data1 = f.read()
email1 = Parser().parsestr(data1)

In [74]:
with open(rootdir2,"r") as f:
    data2 = f.read()
email2 = Parser().parsestr(data2)

Taking two email file for exicuting the program

In [75]:
text1 = email1.get_payload().replace("\n","")
text2 = email2.get_payload().replace("\n","")

In [76]:
text1

'Mr. Lay,  I have received the proposal from Gulstream and will drop it off today.  As you probably already know, the flight from Beijing was a record first for the GV into Houston.I have met and discussed the possibility of the GV replacing a 900B or a total fleet transition at length with John Stabler, Ron Fain and Keith Jones.  I have prepared a spread sheet showing the affect on budget as well as our thoughts and recommendations.  Although the cabin size of the Global Express is larger, Gulfstream is a good two years ahead of the Global in deliveries and fleet size.  They have overcome the mechanical problems that were experienced earlier in the program,  problems that  the Global Express may still be facing.  Worldwide support and operational experience have the Gulfstream far ahead of the Global Express.  Bombardier is still looking forward to demonstrating their aircraft to you on your India trip in December.All of us at the department stand ready to do whatever you feel necessa

In [77]:
stopwords_en = stopwords.words("english")

function to tokenize the the documents

In [78]:
def preprocessing(raw):
    wordlist = nltk.word_tokenize(raw)
    text = [w.lower() for w in wordlist if w not in stopwords_en]
    return text

In [79]:
text1 = preprocessing(text1)
text2 = preprocessing(text2)

creating frequency dictionary for the documents 

In [80]:
word_set = set(text1).union(set(text2))

freqd_text1 = FreqDist(text1)
text1_count_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_count_dict[word] = freqd_text1[word]

freqd_text2 = FreqDist(text2)
text2_count_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_count_dict[word] = freqd_text2[word]

In [81]:
freqd_text2

FreqDist({'.': 6, 'president': 3, 'congress': 3, 'operations': 2, 'authority': 2, 'extend': 2, 'exemption': 2, 'congressional': 2, 'government': 2, 'clinton': 1, ...})

In [82]:
freqd_text1

FreqDist({'.': 7, ',': 5, 'global': 4, 'express': 3, 'i': 2, 'gv': 2, 'fleet': 2, 'size': 2, 'gulfstream': 2, 'ahead': 2, ...})

TF-IDF Method

In [83]:
freqd_text1 = FreqDist(text1)
text1_length = len(text1)
text1_tf_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_tf_dict[word] = freqd_text1[word]/text1_length

freqd_text2 = FreqDist(text2)
text2_length = len(text2)
text2_tf_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_tf_dict[word] = freqd_text2[word]/text2_length

In [84]:
text12_idf_dict = dict.fromkeys(word_set,0)
text12_length = 2

for word in text12_idf_dict.keys():
    if word in text1:
        text12_idf_dict[word] += 1
    if word in text2:
        text12_idf_dict[word] += 1


In [85]:
for word, val in text12_idf_dict.items():
    text12_idf_dict[word] = 1 + math.log(text12_length/(float(val)))

In [86]:
text1_tfidf_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_tfidf_dict[word] = (text1_tf_dict[word]) * (text12_idf_dict[word])

text2_tfidf_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_tfidf_dict[word] = (text2_tf_dict[word]) * (text12_idf_dict[word])

Word Embedding Method

In [87]:
taggeddocs = list()
doc1 = TaggedDocument(words=text1,tags= ["Mail_1"])
taggeddocs.append(doc1)
doc2 = TaggedDocument(words=text2,tags= ["Mail_2"])
taggeddocs.append(doc2)

In [88]:
print(doc1)

TaggedDocument(['mr.', 'lay', ',', 'i', 'received', 'proposal', 'gulstream', 'drop', 'today', '.', 'as', 'probably', 'already', 'know', ',', 'flight', 'beijing', 'record', 'first', 'gv', 'houston.i', 'met', 'discussed', 'possibility', 'gv', 'replacing', '900b', 'total', 'fleet', 'transition', 'length', 'john', 'stabler', ',', 'ron', 'fain', 'keith', 'jones', '.', 'i', 'prepared', 'spread', 'sheet', 'showing', 'affect', 'budget', 'well', 'thoughts', 'recommendations', '.', 'although', 'cabin', 'size', 'global', 'express', 'larger', ',', 'gulfstream', 'good', 'two', 'years', 'ahead', 'global', 'deliveries', 'fleet', 'size', '.', 'they', 'overcome', 'mechanical', 'problems', 'experienced', 'earlier', 'program', ',', 'problems', 'global', 'express', 'may', 'still', 'facing', '.', 'worldwide', 'support', 'operational', 'experience', 'gulfstream', 'far', 'ahead', 'global', 'express', '.', 'bombardier', 'still', 'looking', 'forward', 'demonstrating', 'aircraft', 'india', 'trip', 'december.all

Creating Gensim Model

In [89]:
model = gensim.models.Doc2Vec(taggeddocs,dm=0,alpha=0.025,vector_size=20,min_alpha=0.025,min_count=0)

In [90]:
token_count = sum([len(sentence) for sentence in taggeddocs])

In [91]:
for epoch in range(80):
    if epoch % 20 ==0:
        print("Now training epoch %s" %epoch)
    model.train(taggeddocs,total_examples = token_count, epochs = 80)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60


In [92]:
v1 = list(text1_tfidf_dict.values())
v2 = list(text2_tfidf_dict.values())

similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
print("Similarity Index: {:4.2f} %".format(similarity*100))

Similarity Index: 11.36 %
